In [1]:
!git clone https://github.com/huggingface/transformers.git
!pip install transformers/.
!pip install -r transformers/examples/pytorch/language-modeling/requirements.txt
!pip install -r transformers/examples/pytorch/text-generation/requirements.txt
!pip install datasets evaluate sacrebleu
!pip install git+https://github.com/huggingface/accelerate

Cloning into 'transformers'...
remote: Enumerating objects: 115294, done.
remote: Counting objects: 100% (360/360), done.
remote: Compressing objects: 100% (238/238), done.
remote: Total 115294 (delta 174), reused 208 (delta 94), pack-reused 114934
Receiving objects: 100% (115294/115294), 109.19 MiB | 17.31 MiB/s, done.
Resolving deltas: 100% (85748/85748), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metad

In [2]:
!git clone https://github.com/sirmammingtonham/trading-fours.git

Cloning into 'trading-fours'...
remote: Enumerating objects: 964, done.
remote: Counting objects: 100% (964/964), done.
remote: Compressing objects: 100% (959/959), done.
remote: Total 964 (delta 10), reused 947 (delta 4), pack-reused 0
Receiving objects: 100% (964/964), 20.62 MiB | 17.82 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [79]:
import torch
import datasets
import evaluate
import numpy as np
import pandas as pd
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader

In [6]:
model = AutoModelForCausalLM.from_pretrained('yeeb/distilgpt2_trading-fours')
tokenizer = AutoTokenizer.from_pretrained('yeeb/distilgpt2_trading-fours')

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/334M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/266 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [37]:
test_df = pd.read_csv('/content/trading-fours/data/test.csv')
test_df['ref'] = test_df['text'].apply(lambda ex: ' '.join(ex.split()[100:]))
test_df['prompt'] = test_df['text'].apply(lambda ex: ' '.join(ex.split()[:100]))
# test_ds = load_dataset('csv', data_files={'test': '/content/trading-fours/data/test.csv'})

In [39]:
test_ds = datasets.Dataset.from_pandas(test_df)

In [57]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [93]:
tokenized_ds = test_ds.map(
      lambda ex: tokenizer(ex['prompt'], padding='max_length', truncation=True),
      batched=True,
      remove_columns=['name', 'text']
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [94]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 768)

In [95]:
tokenized_ds.set_format("torch")

In [96]:
test_dataloader = DataLoader(
    tokenized_ds, shuffle=True, batch_size=32
)

In [97]:
metric = evaluate.load('sacrebleu')

In [98]:
def evaluate_function(model, test_dataloader, outputs):
  accelerator = Accelerator()

  model, test_dataloader = accelerator.prepare(
        model, test_dataloader
    )
  model.eval()
  for step, batch in enumerate(test_dataloader):
    print(step)
    with torch.no_grad():
      generated_tokens = model.generate(
          batch["input_ids"],
          attention_mask=batch["attention_mask"],
          max_length=tokenizer.model_max_length
      )
      refs = batch["ref"]

      generated_tokens = generated_tokens.cpu().numpy()
      # refs = refs.cpu().numpy()
      decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
      # decoded_refs = tokenizer.decode(refs, skip_special_tokens=True)

      metric.add_batch(predictions=decoded_preds, references=refs)
      outputs.append({'prompt': batch['prompt'], 'generated': decoded_preds})
  eval_metric = metric.compute()
  print({"bleu": eval_metric["score"]})


In [99]:
from accelerate import notebook_launcher

outputs = []
notebook_launcher(evaluate_function, (model,test_dataloader,outputs))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1024, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Launching training on one GPU.
0


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1024, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


1


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1024, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


2


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1024, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


3


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1024, but `max_length` is set to 1024. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


4
{'bleu': 0.0254594839252388}


In [102]:
prompts = []
gens = []
for i in range(len(outputs)):
  for prompt, gen in zip(outputs[i]['prompt'], outputs[i]['generated']):
    prompts.append(prompt)
    gens.append(gen)

In [106]:
df = pd.DataFrame(list(zip(prompts, gens)), columns=['prompt', 'generation'])
df.to_csv('/content/distilgpt2-test_output.csv')
df.head()

,prompt,generation
0,A#1 A#0 A1 G#2 G#1 A1 A#1 A#1 A1 G#2 G2 G2 C2 ...,A#1 A#0 A1 G#2 G#1 A1 A#1 A#1 A1 G#2 G2 G2 C2 ...
1,D1 D1 D1 A0 D1 E1 F1 F#1 G1 G1 G1 D1 G1 D1 G1 ...,D1 D1 D1 A0 D1 E1 F1 F#1 G1 G1 G1 D1 G1 D1 G1 ...
2,G#3 A3 B3 B3 C#4 D4 E4 B3 C#4 D4 D#4 C#4 C4 A#...,G#3 A3 B3 B3 C#4 D4 E4 B3 C#4 D4 D#4 C#4 C4 A#...
3,B1 B2 E4 B3 G#3 C#4 A3 G#3 B3 G#3 E3 B3 E2 F#2...,B1 B2 E4 B3 G#3 C#4 A3 G#3 B3 G#3 E3 B3 E2 F#2...
4,E3 G#2 E4 B3 G#3 E3 D#4 B3 F#3 B3 C#4 D#4 E4 B...,E3 G#2 E4 B3 G#3 E3 D#4 B3 F#3 B3 C#4 D#4 E4 B...
